### Querying current flight positions from OpenSky

In [1]:
from urllib import request

with request.urlopen("https://opensky-network.org/api/states/all") as response:
    print(response.read().decode("utf-8"))

{"time":1733269358,"states":[["a545d8","N439PF  ","United States",1733269357,1733269358,-115.2839,36.1794,13106.4,false,211.98,298.24,0,null,13296.9,"2560",false,0],["ab1644","UAL2447 ","United States",1733269357,1733269357,-99.7518,34.5647,11582.4,false,207.88,338.51,0,null,11910.06,"7210",false,0],["a798d1","N589RC  ","United States",1733269357,1733269358,-123.0583,45.336,845.82,false,79.81,348.47,0,null,922.02,null,false,0],["ae1fa0","72209   ","United States",1733269357,1733269357,-85.5176,37.694,754.38,false,55.83,106.6,0,null,830.58,"4265",false,0],["aa3cbe","N759PA  ","United States",1733269357,1733269357,-111.873,41.7829,1569.72,false,52.02,182.83,-0.65,null,1676.4,null,false,0],["a5cb05","","United States",null,1733269190,null,null,null,false,34.07,284.88,-1.3,null,null,null,false,0],["a2e5ec","SKW3693 ","United States",1733269357,1733269357,-118.5685,34.3703,1783.08,false,145.74,204.39,0.33,null,1836.42,"7746",false,0],["aa56db","UAL1230 ","United States",1733269357,173326935

### Filtering flight positions from OpenSky

In [2]:
import json
from urllib import request

with request.urlopen("https://opensky-network.org/api/states/all?lamin=45.8389&lomin=5.9962&lamax=47.8229&lomax=10.5226") as response:
    print(json.loads(response.read().decode("utf-8")))

{'time': 1733269545, 'states': [['4b1807', 'SWR     ', 'Switzerland', 1733269538, 1733269544, 8.557, 47.4536, 358.14, True, 0, 5.62, None, None, None, '2000', False, 0], ['4b180b', 'SWR     ', 'Switzerland', 1733269532, 1733269543, 8.5586, 47.4535, 358.14, True, 0, 5.62, None, None, None, '2000', False, 0], ['3c5421', 'BCS9TC  ', 'Germany', 1733269545, 1733269545, 7.9709, 47.7685, 10363.2, False, 228.86, 33.73, 0, None, 10279.38, '1000', False, 0], ['3c1bb0', 'BCS4HG  ', 'Germany', 1733269544, 1733269544, 10.1895, 46.0849, 9654.54, False, 233.87, 41.97, 4.88, None, 9563.1, '1000', False, 0], ['4074e4', 'BAW157  ', 'United Kingdom', 1733269544, 1733269544, 7.5044, 47.56, 11887.2, False, 272.27, 120.93, -0.33, None, 11803.38, '2267', False, 0], ['010262', 'MSR780  ', 'Egypt', 1733269544, 1733269544, 9.795, 46.58, 10668, False, 245.09, 119.42, -0.33, None, 10561.32, '5241', False, 0], ['347302', 'BCS408  ', 'Spain', 1733269544, 1733269544, 8.7903, 47.3055, 10370.82, False, 219.23, 351.09,

### Creating events from flight data

In [3]:
def response_to_events(api_response):
    flight_events = []
    for update in api_response["states"]:
        flight_events.append(
            {
                "icao24": update[0],
                "origin_country": update[2],
                "time_position": update[3],
                "longitude": update[5],
                "latitude": update[6],
                "velocity": update[9],
                "true_track": update[10],
            }
        )
    return sorted(flight_events, key=lambda x: x["time_position"])

### A flight update event generator

In [4]:
import time
from urllib import request

def get_events(
    url="https://opensky-network.org/api/states/all?lamin=45.8389&lomin=5.9962&lamax=47.8229&lomax=10.5226",
    interval_sec=10,
):
    for _ in range(3):
        with request.urlopen(url) as response:
            yield from response_to_events(json.loads(response.read().decode("utf-8")))
        time.sleep(interval_sec)

### Calling the event generator

In [5]:
for event in get_events():
    print(event)

{'icao24': '4b43ad', 'origin_country': 'Switzerland', 'time_position': 1733269550, 'longitude': 6.9466, 'latitude': 46.8184, 'velocity': 2.3, 'true_track': 63.43}
{'icao24': '4b5d70', 'origin_country': 'Switzerland', 'time_position': 1733269750, 'longitude': 8.5588, 'latitude': 47.4601, 'velocity': 12.86, 'true_track': 196.88}
{'icao24': '4b5d8b', 'origin_country': 'Switzerland', 'time_position': 1733269753, 'longitude': 8.5588, 'latitude': 47.4601, 'velocity': 10.8, 'true_track': 208.12}
{'icao24': '4b1807', 'origin_country': 'Switzerland', 'time_position': 1733269757, 'longitude': 8.557, 'latitude': 47.4536, 'velocity': 0, 'true_track': 5.62}
{'icao24': '4b180b', 'origin_country': 'Switzerland', 'time_position': 1733269762, 'longitude': 8.5586, 'latitude': 47.4535, 'velocity': 0, 'true_track': 5.62}
{'icao24': '4b17e3', 'origin_country': 'Switzerland', 'time_position': 1733269764, 'longitude': 8.5575, 'latitude': 47.4518, 'velocity': 0, 'true_track': 185.62}
{'icao24': '407b70', 'ori